In [10]:
import json
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

def load_json_file(filename):
    """Carga un archivo JSON y maneja posibles errores."""
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo {filename}")
        return None
    except json.JSONDecodeError:
        print(f"Error: El archivo {filename} no es un JSON válido")
        return None
    except Exception as e:
        print(f"Error inesperado al cargar {filename}: {str(e)}")
        return None

def load_all_data():
    """Carga todos los archivos JSON necesarios."""
    data = {
        'horarios_salas': load_json_file('../../agent_output/Horarios_salas.json'),
        'salas_input': load_json_file('../../agent_input/InputOfSala.json')
    }
    
    if None in data.values():
        print("Error: No se pudieron cargar todos los archivos necesarios")
        return None
    return data

def create_room_capacity_dict(salas_input):
    """Crea un diccionario con las capacidades de las salas."""
    try:
        return {room['Codigo']: room['Capacidad'] for room in salas_input}
    except KeyError as e:
        print(f"Error: Falta la clave {e} en los datos de las salas")
        return None

def categorize_capacity(current_capacity):
    """Categoriza la capacidad según los rangos definidos."""
    if current_capacity < 1.0:
        return 'bajo'
    elif current_capacity > 1.00:
        return 'sobre'
    else:
        return 'exacta'

def analyze_room_assignments(horarios_salas, room_capacities):
    """Analiza las asignaciones de cada sala."""
    capacity_stats = defaultdict(lambda: {'bajo': 0, 'exacta': 0, 'sobre': 0, 'total': 0})
    
    try:
        for sala in horarios_salas:
            codigo = sala['Codigo']
            
            for asignatura in sala['Asignaturas']:
                current_capacity = asignatura['Capacidad']
                capacity_stats[codigo]['total'] += 1
                category = categorize_capacity(current_capacity)
                capacity_stats[codigo][category] += 1
                
        return capacity_stats
    except KeyError as e:
        print(f"Error: Falta la clave {e} en los datos de asignaciones")
        return None

def create_capacity_dataframe(capacity_stats, room_capacities):
    """Crea un DataFrame con las estadísticas de capacidad."""
    try:
        df_stats = pd.DataFrame.from_dict(capacity_stats, orient='index')
        df_stats['room_capacity'] = df_stats.index.map(room_capacities)
        
        # Calcular porcentajes
        for col in ['bajo', 'exacta', 'sobre']:
            df_stats[f'{col}_pct'] = (df_stats[col] / df_stats['total'] * 100).round(2)
        
        # Crear rangos de capacidad
        df_stats['capacity_range'] = pd.cut(
            df_stats['room_capacity'], 
            bins=range(0, 91, 10), 
            labels=[f'{i}-{i+9}' for i in range(0, 81, 10)]
        )
        
        return df_stats
    except Exception as e:
        print(f"Error al crear el DataFrame: {str(e)}")
        return None

def create_summary_table(df_stats):
    """Crea una tabla resumen de las estadísticas de capacidad."""
    try:
        summary_table = pd.DataFrame({
            'Sala': df_stats.index,
            'Capacidad Total': df_stats['room_capacity'],
            'Bajo Capacidad (%)': df_stats['bajo_pct'],
            'Capacidad Exacta (%)': df_stats['exacta_pct'],
            'Sobre Capacidad (%)': df_stats['sobre_pct']
        })
        
        # Guardar tabla en CSV
        summary_table.to_csv('resumen_capacidad.csv', index=False)
        
        # Mostrar tabla formateada
        print("\nResumen de Capacidad por Sala:")
        print(summary_table.to_string(index=False))
        
        return summary_table
    except Exception as e:
        print(f"Error al crear la tabla resumen: {str(e)}")
        return None

def create_mix_chart(df_stats):
    """Crea un gráfico combinado: donut chart y lateral bar chart."""
    try:
        # Calcular totales para las categorías
        total_counts = df_stats[['bajo', 'exacta', 'sobre']].sum()
        total_blocks = total_counts.sum()
        
        # Configurar el subplot con 1 fila y 2 columnas
        fig = plt.figure(figsize=(15, 6))
        
        # Crear el donut chart (izquierda)
        ax1 = plt.subplot(121)
        wedges, texts, autotexts = ax1.pie(total_counts, 
                labels=['Asientos Disponibles', 'Asientos justos', 'Asientos Insuficientes'],
                colors=['#99C2E8', '#00629B', '#A9A9A9'],  # Colores IEEE
                autopct='%1.1f%%',
                pctdistance=0.75)  # Ajustar la distancia de los porcentajes

        # Ajustar el tamaño y la posición de los porcentajes
        for i, autotext in enumerate(autotexts):
            if i == 0:  # Asientos Disponibles
                autotext.set_color('black')
            elif i == 1:  # Asientos justos
                autotext.set_color('white')
            else:  # Asientos Insuficientes
                autotext.set_color('black')
            autotext.set_fontsize(12)  # Cambiar el tamaño de la fuente
        
        # Crear el agujero central del donut
        centre_circle = plt.Circle((0,0), 0.50, fc='white')
        ax1.add_artist(centre_circle)
        
        # Agregar texto en el centro (suma de Bajo y Exacta)
        suma_porcentual = ((total_counts['bajo'] + total_counts['exacta']) / total_blocks * 100)
        plt.text(0, 0, f'{suma_porcentual:.1f}%', 
                ha='center', va='center', fontsize=20)
        
        ax1.set_title('Disponibilidad de Asientos Frente a la Asignacion de Salas')
        
        # Crear el bar chart horizontal (derecha)
        ax2 = plt.subplot(122)
        categories = ['Salas con Asientos Disponibles', 'Salas con Asientos Justos', 'Salas con Asientos Insuficientes']
        colors = ['#99C2E8', '#00629B', '#A9A9A9']

        # Create paired lists and sort them
        zipped = list(zip(total_counts, categories, colors))
        sorted_pairs = sorted(zipped, reverse=True)  # Sort in descending order
        total_counts_sorted, categories_sorted, colors_sorted = zip(*sorted_pairs)
        y_pos = range(len(categories))

        # Eliminar el borde superior y derecho
        ax2.spines['top'].set_visible(False)
        ax2.spines['right'].set_visible(False)

        # Create horizontal bars with sorted data
        ax2.barh(y_pos, total_counts_sorted, color=colors_sorted)

        # Invert y-axis to show larger values on top
        ax2.invert_yaxis()

        # Update labels with sorted categories
        ax2.set_yticks(y_pos)
        ax2.set_yticklabels(categories_sorted)
        ax2.yaxis.set_tick_params(pad=10)  # Ajustar la posición de las etiquetas
        ax2.set_title(f'Distribución de Asignaturas por Disponibilidad de Asientos\n(Total: {int(total_blocks)} asignaturas)')

        # Add values at end of bars
        for i, v in enumerate(total_counts_sorted):
            ax2.text(v, i, f' {int(v)}', va='center')

        # Agregar una línea divisoria entre los gráficos
        fig.subplots_adjust(wspace=0.5)  # Ajustar el Asientos entre subplots
        
        plt.tight_layout()
        plt.savefig('capacidad_mixta.png')
        plt.close()
        
        return total_counts
        
    except Exception as e:
        print(f"Error al crear el gráfico mixto: {str(e)}")
        return None

def create_descriptive_statistics(summary_table):
    try:
        # Estadísticas para Capacidad Total
        capacidad_stats = {
            'Métrica': [
                'Número de Salas',
                'Capacidad Mínima',
                'Primer Cuartil (Q1)',
                'Mediana',
                'Tercer Cuartil (Q3)',
                'Capacidad Máxima',
                'Rango Intercuartílico'
            ],
            'Valor': [
                len(summary_table),
                summary_table['Capacidad Total'].min(),
                summary_table['Capacidad Total'].quantile(0.25),
                summary_table['Capacidad Total'].median(),
                summary_table['Capacidad Total'].quantile(0.75),
                summary_table['Capacidad Total'].max(),
                summary_table['Capacidad Total'].quantile(0.75) - summary_table['Capacidad Total'].quantile(0.25)
            ]
        }
        
        # Análisis de distribución de capacidad
        ranges = [(0,20), (21,40), (41,60), (61,80), (81,100)]
        capacity_dist = []
        for start, end in ranges:
            count = len(summary_table[
                (summary_table['Capacidad Total'] >= start) & 
                (summary_table['Capacidad Total'] <= end)
            ])
            capacity_dist.append(f"Salas con {start}-{end} asientos: {count}")
        
        # Crear DataFrames
        df_stats = pd.DataFrame(capacidad_stats)
        
        # Guardar en CSV
        df_stats.to_csv('descriptcion_salas.csv', index=False)
        
        # Mostrar resultados
        print("\nDescripcion de Capacidad:")
        print(df_stats.to_string(index=False))
        print("\nDistribución de Capacidad:")
        for dist in capacity_dist:
            print(dist)
            
        return df_stats
        
    except Exception as e:
        print(f"Error al crear descripcion: {str(e)}")
        return None

def save_results(df_stats, total_counts, range_analysis):
    """Guarda los resultados en un archivo JSON."""
    try:
        results = {
            'room_stats': df_stats.to_dict(orient='index'),
            'global_stats': {
                'bajo_capacidad': float(total_counts['bajo'] / total_counts.sum() * 100),
                'capacidad_exacta': float(total_counts['exacta'] / total_counts.sum() * 100),
                'sobre_capacidad': float(total_counts['sobre'] / total_counts.sum() * 100)
            },
            'range_analysis': range_analysis.to_dict()
        }
        
        with open('metricas_capacidad.json', 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
        
        return True
    except Exception as e:
        print(f"Error al guardar los resultados: {str(e)}")
        return False

def main():
    # 1. Cargar datos
    data = load_all_data()
    if data is None:
        return
    
    # 2. Crear diccionario de capacidades
    room_capacities = create_room_capacity_dict(data['salas_input'])
    if room_capacities is None:
        return
    
    # 3. Analizar asignaciones
    capacity_stats = analyze_room_assignments(data['horarios_salas'], room_capacities)
    if capacity_stats is None:
        return
    
    # 4. Crear DataFrame
    df_stats = create_capacity_dataframe(capacity_stats, room_capacities)
    if df_stats is None:
        return
    
    # 5. Crear tabla resumen
    summary_table = create_summary_table(df_stats)
    if summary_table is None:
        return
    
    # 5.1 Crear estadísticas descriptivas
    descriptive_stats = create_descriptive_statistics(summary_table)
    if descriptive_stats is None:
        return
    
    range_analysis = create_mix_chart(df_stats)
    if range_analysis is None:
        return
    
    # 7. Guardar resultados
    if save_results(df_stats, total_counts, range_analysis):
        print("\nAnálisis completado exitosamente")
    else:
        print("Error al completar el análisis")

if __name__ == "__main__":
    main()


Resumen de Capacidad por Sala:
 Sala  Capacidad Total  Bajo Capacidad (%)  Capacidad Exacta (%)  Sobre Capacidad (%)
KAUS3               26               33.33                 66.67                 0.00
KAUS2               26              100.00                  0.00                 0.00
KAUS1               68              100.00                  0.00                 0.00
   K1               82              100.00                  0.00                 0.00
   K2               30               75.00                 25.00                 0.00
 A106               18               48.89                  4.44                46.67
  CM3               50              100.00                  0.00                 0.00
   K3               40              100.00                  0.00                 0.00
  CM5               45               96.67                  3.33                 0.00
  CM4               45               96.67                  3.33                 0.00
   E1               45

NameError: name 'total_counts' is not defined